## <span style="color:#ff5f27">🧬 Train Retrieval Model </span>

In this notebook, we will train a retrieval model that will be able to quickly generate a small subset of candidate items from a large collection of items. Our model will be based on the two-tower architecture, which embeds queries and candidates (keys) into a shared low-dimensional vector space. Here, a query consists of features of a customer and a transaction (e.g. timestamp of the purchase), whereas a candidate consists of features of a particular item. All queries will have a user ID and all candidates will have an item ID, and the model will be trained such that the embedding of a user will be close to all the embeddings of items the user has previously bought.

After training the model we will save and upload its components to the Hopsworks Model Registry.

Let's go ahead and load the data.

## <span style="color:#ff5f27">📝 Imports </span>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import StringLookup, Normalization

import warnings
warnings.filterwarnings('ignore')

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## <span style="color:#ff5f27">🔪 Feature Selection </span>

First, we'll load the feature groups we created in the previous tutorial.

In [ ]:
trans_fg = fs.get_feature_group(
    name="transactions",
    version=1,
)
customers_fg = fs.get_feature_group(
    name="customers",
    version=1,
)
articles_fg = fs.get_feature_group(
    name="articles",
    version=1,
)

We'll need to join these three data sources to make the data compatible with out retrieval model. Recall that each row in the `transactions` feature group relates information about which customer bought which item. We'll join this feature group with the `customers` and `articles` feature groups to inject customer and item features into each row.

In [ ]:
query = trans_fg.select(["customer_id", "article_id", "t_dat", "price", "month_sin", "month_cos"])\
    .join(customers_fg.select(["age", "club_member_status", "age_group"]), on="customer_id")\
    .join(articles_fg.select(["garment_group_name", "index_group_name"]), on="article_id")

## <span style="color:#ff5f27">⚙️ Feature View Creation </span>

In Hopsworks, you write features to feature groups (where the features are stored) and you read features from feature views. A feature view is a logical view over features, stored in feature groups, and a feature view typically contains the features used by a specific model. This way, feature views enable features, stored in different feature groups, to be reused across many different models.

In [ ]:
feature_view = fs.get_or_create_feature_view(
    name='retrieval',
    query=query,
    version=1,
)

To view and explore data in the feature view you can retrieve batch data using the `get_batch_data()` method.

## <span style="color:#ff5f27">🏋️ Training Dataset </span>


In [ ]:
train_df, val_df, test_df, y_train, y_val, y_test = feature_view.train_validation_test_split(
    validation_size=0.1, 
    test_size=0.1,
    description='Retrieval dataset splits',
)

We will train our retrieval model with a subset of features.

For the query embedding we will use:
- `customer_id`: ID of the customer.
- `age`: age of the customer at the time of purchase.
- `month_sin`, `month_cos`: time of year the purchase was made.

For the candidate embedding we will use:
- `article_id`: ID of the item.
- `garment_group_name`: type of garment.
- `index_group_name`: menswear/ladieswear etc.

In [ ]:
train_df["article_id"] = train_df["article_id"].astype(str)  # to be removed
val_df["article_id"] = val_df["article_id"].astype(str)  # to be removed

In [ ]:
query_features = ["customer_id", "age", "month_sin", "month_cos"]
candidate_features = ["article_id", "garment_group_name", "index_group_name"]

def df_to_ds(df):
    return tf.data.Dataset.from_tensor_slices({col : df[col] for col in df})

BATCH_SIZE = 2048
train_ds = df_to_ds(train_df).batch(BATCH_SIZE).cache().shuffle(BATCH_SIZE*10)
val_ds = df_to_ds(val_df).batch(BATCH_SIZE).cache()

You will need a list of user and item IDs when we initialize our embeddings.

In [ ]:
user_id_list = train_df["customer_id"].unique().tolist()
item_id_list = train_df["article_id"].unique().tolist()

garment_group_list = train_df["garment_group_name"].unique().tolist()
index_group_list = train_df["index_group_name"].unique().tolist()

print(f"Number of transactions: {len(train_df):,}")
print(f"Number of users: {len(user_id_list):,}")
print(f"Number of items: {len(item_id_list):,}")
print(garment_group_list)

## <span style="color:#ff5f27">🏰 Two Tower Model </span>

The two tower model consist of two models:
- Query model: generates a query representation given user and transaction features.
- Candidate model: generates an item representation given item features.

Both models produce embeddings that live in the same embedding space. We let this space be low-dimensional to prevent overfitting on the training data. (Otherwise, the model might simply memorize previous purchases, which makes it recommend items customers already have bought.)

In [ ]:
EMB_DIM = 16

We start with creating the query model.

In [ ]:
class QueryTower(tf.keras.Model):

    def __init__(self):
        super().__init__()

        self.user_embedding = tf.keras.Sequential([
            StringLookup(
                vocabulary=user_id_list,
                mask_token=None
            ),
            tf.keras.layers.Embedding(
                # We add an additional embedding to account for unknown tokens.
                len(user_id_list) + 1,
                EMB_DIM
            )
        ])

        self.normalized_age = Normalization(axis=None)

        self.fnn = tf.keras.Sequential([
            tf.keras.layers.Dense(EMB_DIM, activation="relu"),
            tf.keras.layers.Dense(EMB_DIM)
        ])

    def call(self, inputs):
        concatenated_inputs = tf.concat([
            self.user_embedding(inputs["customer_id"]),
            tf.reshape(self.normalized_age(inputs["age"]), (-1,1)),
            tf.reshape(inputs["month_sin"], (-1,1)),
            tf.reshape(inputs["month_cos"], (-1,1))
        ], axis=1)

        outputs = self.fnn(concatenated_inputs)

        return outputs


query_model = QueryTower()

query_model.normalized_age.adapt(train_ds.map(lambda x : x["age"]))

# Initialize model with inputs.
query_df = train_df[query_features]
query_ds = df_to_ds(query_df).batch(1)
query_model(next(iter(query_ds)))

The candidate model is very similar to the query model. A difference is that it has two categorical features as input, which we one-hot encode.

In [ ]:
class ItemTower(tf.keras.Model):

    def __init__(self):
        super().__init__()

        self.item_embedding = tf.keras.Sequential([
            StringLookup(
                vocabulary=item_id_list,
                mask_token=None
            ),
            tf.keras.layers.Embedding(
                # We add an additional embedding to account for unknown tokens.
                len(item_id_list) + 1,
                EMB_DIM
            )
        ])

        self.garment_group_tokenizer = StringLookup(vocabulary=garment_group_list, mask_token=None)
        self.index_group_tokenizer = StringLookup(vocabulary=index_group_list, mask_token=None)

        self.fnn = tf.keras.Sequential([
            tf.keras.layers.Dense(EMB_DIM, activation="relu"),
            tf.keras.layers.Dense(EMB_DIM)
        ])

    def call(self, inputs):
        garment_group_embedding = tf.one_hot(
            self.garment_group_tokenizer(inputs["garment_group_name"]),
            len(garment_group_list)
        )

        index_group_embedding = tf.one_hot(
            self.index_group_tokenizer(inputs["index_group_name"]),
            len(index_group_list)
        )

        concatenated_inputs = tf.concat([
            self.item_embedding(inputs["article_id"]),
            garment_group_embedding,
            index_group_embedding
        ], axis=1)

        outputs = self.fnn(concatenated_inputs)

        return outputs


item_model = ItemTower()

You will evaluate the two tower model using the *top-100 accuracy*. That is, for each transaction in the validation data we will generate the associated query embedding and retrieve the set of the 100 items that are closest to this query in the embedding space. The top-100 accuracy measures how often the item that was actually bought is part of this subset. To evaluate this, we create a dataset of all unique items in the training data.

In [ ]:
item_df = train_df[candidate_features]
item_df.drop_duplicates(subset="article_id", inplace=True)
item_ds = df_to_ds(item_df)

With this in place, we can finally create our two tower model.

In [ ]:
import tensorflow_recommenders as tfrs

class TwoTowerModel(tf.keras.Model):
    def __init__(self, query_model, item_model):
        super().__init__()
        self.query_model = query_model
        self.item_model = item_model
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=item_ds.batch(BATCH_SIZE).map(self.item_model)
            )
        )

    def train_step(self, batch) -> tf.Tensor:
        # Set up a gradient tape to record gradients.
        with tf.GradientTape() as tape:

            # Loss computation.
            user_embeddings = self.query_model(batch)
            item_embeddings = self.item_model(batch)
            loss = self.task(
                user_embeddings, 
                item_embeddings,
                compute_metrics=False,
            )

            # Handle regularization losses as well.
            regularization_loss = sum(self.losses)

            total_loss = loss + regularization_loss

        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        metrics = {
            "loss": loss,
            "regularization_loss": regularization_loss,
            "total_loss": total_loss
        }

        return metrics

    def test_step(self, batch) -> tf.Tensor:
        # Loss computation.
        user_embeddings = self.query_model(batch)
        item_embeddings = self.item_model(batch)

        loss = self.task(
            user_embeddings, 
            item_embeddings,
            compute_metrics=False,
        )

        # Handle regularization losses as well.
        regularization_loss = sum(self.losses)

        total_loss = loss + regularization_loss

        metrics = {metric.name: metric.result() for metric in self.metrics}
        metrics["loss"] = loss
        metrics["regularization_loss"] = regularization_loss
        metrics["total_loss"] = total_loss

        return metrics

### <span style="color:#ff5f27">🏃🏻‍♂️ Model Training </span>

You'll train our model using the AdamW optimizer, which applies weight regularization during training.

In [ ]:
import tensorflow_addons as tfa

model = TwoTowerModel(query_model, item_model)
optimizer = tfa.optimizers.AdamW(0.001, learning_rate=0.01)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=5)

## <span style="color:#ff5f27">🗄️ Upload Model to Model Registry </span>

One of the features in Hopsworks is the model registry. This is where we can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

Let's connect to the model registry using the [HSML library](https://docs.hopsworks.ai/machine-learning-api/latest) from Hopsworks.

In [ ]:
mr = project.get_model_registry()

In [ ]:
class QueryModelModule(tf.Module):
    def __init__(self, query_model):
        self.query_model = query_model

    @tf.function()
    def compute_emb(self, instances):
        query_emb = self.query_model(instances)
        return {"customer_id": instances["customer_id"],
                "month_sin": instances["month_sin"],
                "month_cos": instances["month_cos"],
                "query_emb": query_emb}

# wrap query_model:   query_model -> query_model_module
query_model = QueryModelModule(model.query_model)

In [ ]:
instances_spec={
    'customer_id': tf.TensorSpec(shape=(None,), dtype=tf.string, name='customer_id'),
    'month_sin': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='month_sin'),
    'month_cos': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='month_cos'),
    'age': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='age')
}
signatures = query_model.compute_emb.get_concrete_function(instances_spec)

tf.saved_model.save(query_model, "query_model", signatures=signatures)

First, we need to save our models locally.

In [ ]:
tf.saved_model.save(model.item_model, "candidate_model")

Each model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/machine-learning-api/latest/generated/model_schema/), which describes the inputs and outputs for a model. A schema can either be manually specified or inferred from data.

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Infer input schema from data.
query_model_input_schema = Schema(query_df)

# Manually specify output schema.
query_model_output_schema = Schema([{
    "name": "query_embedding",
    "type": "float32",
    "shape": [EMB_DIM]
}])

query_model_schema = ModelSchema(
    input_schema=query_model_input_schema,
    output_schema=query_model_output_schema
)

query_model_schema.to_dict()

With the schema in place, we can finally register our model.

In [ ]:
query_example = query_df.sample().to_dict("records")

mr_query_model = mr.tensorflow.create_model(
    name="query_model",
    description="Model that generates query embeddings from user and transaction features",
    input_example=query_example,
    model_schema=query_model_schema,
)
mr_query_model.save("query_model")

Here we have also saved an input example from the training data, which can be helpful for test purposes.

Let's repeat the process with the candidate model.

In [ ]:
candidate_model_input_schema = Schema(item_df)

candidate_model_output_schema = Schema([{
    "name": "candidate_embedding",
    "type": "float32",
    "shape": [EMB_DIM],
}])

candidate_model_schema = ModelSchema(
    input_schema=candidate_model_input_schema,
    output_schema=candidate_model_output_schema
)

candidate_example = item_df.sample().to_dict("records")

mr_candidate_model = mr.tensorflow.create_model(
    name="candidate_model",
    description="Model that generates candidate embeddings from item features",
    input_example=candidate_example,
    model_schema=candidate_model_schema,
)
mr_candidate_model.save("candidate_model")

---
## <span style="color:#ff5f27">⏩️ Next Steps </span>

Retrieving the top-k closest candidate embeddings in a brute-force way (computing the distances between the query embedding and all candidate embeddings) is too expensive in a practical setting. In the next notebook, we will index the item embeddings using OpenSearch, which will allow us to retrieve candidates with very low latency.